In [20]:
# import secrets and tokens from config.py
from config import client_id, client_secret, refresh_token

import requests
import urllib3

# import packages for data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

In [ ]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

In [22]:
activities = pd.json_normalize(my_dataset)

In [23]:
#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date

In [24]:
# Make a copy of activities DataFrame for testing.
activities_copy = activities.copy()

In [25]:
# convert data types
activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)
# convert values
activities_copy.loc[:, 'distance'] /= 1609.344 # convert from meters to miles
activities_copy.loc[:, 'average_speed'] *= 2.23693629 # convert from meters/second to miles/hour
activities_copy.loc[:, 'max_speed'] *= 2.23693629 # convert from meters/second to miles/hour
# set index
#activities_copy.set_index('start_date_local', inplace=True)

C:\Users\jluong\AppData\Local\Temp\ipykernel_72904\1035570082.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
C:\Users\jluong\AppData\Local\Temp\ipykernel_72904\1035570082.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)


In [26]:
# df.query('column_name.str.contains("abc")', engine='python')
activities_nfs_copy = activities_copy.copy(deep = True)
# the line below is likely causing the "A value is trying to be set on a copy of a slice from a DataFrame." warning.
activities_nfs = activities_nfs_copy.query('name.str.contains("NFS")', engine = 'python')
len(activities_nfs)

4

In [27]:
# import modules
import os
import time
import matplotlib.pyplot as plt
import folium
import polyline
import base64
from tqdm import tqdm

In [28]:
activities_nfs['map.summary_polyline'].head(3)

5     kgjjF~hrdVkP{FcCs@SBGN]~BEHsBg@I@wBvLMX_@VIRq@...
12    unjjFnerdVsMgE_Ck@qDfSKPY@M^o@pEO`@iGz^}CdQc@|...
26    aijjFxgrdVaD_A{Q_Ge@IONkBpLkA~FcDdRwEbXgCbOS`A...
Name: map.summary_polyline, dtype: object

In [32]:
activities_nfs['map.polyline'] = activities_nfs['map.summary_polyline'].apply(polyline.decode)


C:\Users\jluong\AppData\Local\Temp\ipykernel_72904\2395662333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_nfs['map.polyline'] = activities_nfs['map.summary_polyline'].apply(polyline.decode)


In [31]:
activities_nfs['map.polyline'].head(2)


5     [(38.56006, -121.50432), (38.56284, -121.50306...
12    [(38.56123, -121.50376), (38.56357, -121.50276...
Name: map.polyline, dtype: object

In [30]:
# add decoded summary polylines
# activities['map.summary_polyline'] contains an encoded polyline
# .apply(polyline.decode) decodes that polyline into latitude and longitude
activities_nfs['map.polyline'] = activities_nfs['map.summary_polyline'].apply(polyline.decode)
# print("turkey")
#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities_nfs.index):
    activity = activities_nfs.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

C:\Users\jluong\AppData\Local\Temp\ipykernel_72904\3895680293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_nfs['map.polyline'] = activities_nfs['map.summary_polyline'].apply(polyline.decode)
  0%|          | 0/4 [00:00<?, ?it/s]


SSLError: HTTPSConnectionPool(host='api.open-elevation.com', port=443): Max retries exceeded with url: /api/v1/lookup?locations=38.56006%2C-121.50432 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:992)')))